#### source: https://github.com/adventuresinML/adventures-in-ml-code/blob/master/gensim_word2vec.py

In [ ]:
# import sys
# !{sys.executable} -m pip install numpy pandas matplotlib sklearn seaborn
# !{sys.executable} -m pip install --upgrade gensim

In [1]:
import logging
import collections
import os
import zipfile

import numpy as np
import gensim
import tensorflow as tf

from gensim.models import word2vec
from tensorflow.keras import Sequential
from tensorflow.keras.preprocessing  import sequence
from tensorflow.keras.preprocessing.sequence import skipgrams
from tensorflow.keras.layers import Input, Dense, Embedding, Dot, Reshape, GlobalAveragePooling1D
from tensorflow.keras.models import Model

from urllib import request

vector_dim = 300

def maybe_download(filename, url, expected_bytes):
    """Download a file if not present, and make sure it's the right size."""
    if not os.path.exists(filename):
        filename, _ = request.urlretrieve(url + filename, filename)
    statinfo = os.stat(filename)
    if statinfo.st_size == expected_bytes:
        print('Found and verified', filename)
    else:
        print(statinfo.st_size)
        raise Exception(
            'Failed to verify ' + filename + '. Can you get to it with a browser?')
    return filename

# Read the data into a list of strings.
def read_data(filename):
    """Extract the first file enclosed in a zip file as a list of words."""
    with zipfile.ZipFile(filename) as f:
        data = f.read(f.namelist()[0]).split()
    return data

def convert_data_to_index(string_data, wv):
    index_data = []
    for word in string_data:
        if word in wv:
            index_data.append(wv.vocab[word].index)
    return index_data

def collect_data():
    url = 'http://mattmahoney.net/dc/'
    filename = maybe_download('text8.zip', url, 31344016)
    if not os.path.exists(filename.strip('.zip')):
        zipfile.ZipFile(filename).extractall()
    return filename

In [2]:
def train(model_name="word2vec-gensim.model"):
    filename = collect_data()
    
    sentences = word2vec.Text8Corpus(filename.strip('.zip'))
    logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
    model = word2vec.Word2Vec(sentences, iter=10, min_count=10, size=300, workers=4)

    str_data = read_data(filename)
    index_data = convert_data_to_index(str_data, model.wv)
    print(str_data[:4], index_data[:4])
    
    model.save(model_name)
    return model

In [3]:
def create_embedding_matrix(model):
    # convert the wv word vectors into a numpy matrix that is suitable for insertion
    # into our TensorFlow and Keras models
    embedding_matrix = np.zeros((len(model.wv.vocab), vector_dim))
    for i in range(len(model.wv.vocab)):
        embedding_vector = model.wv[model.wv.index2word[i]]
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    return embedding_matrix

def keras_model(embedding_matrix, wv):
    valid_size = 16  # Random set of words to evaluate similarity on.
    valid_window = 100  # Only pick dev samples in the head of the distribution.
    valid_examples = np.random.choice(valid_window, valid_size, replace=False)
    
    # input words - in this case we do sample by sample evaluations of the similarity
    valid_word = Input((1,), dtype='int32')
    other_word = Input((1,), dtype='int32')
    
    # setup the embedding layer
    embeddings = Embedding(input_dim=embedding_matrix.shape[0], output_dim=embedding_matrix.shape[1], weights=[embedding_matrix])
    embedded_a = embeddings(valid_word)
    embedded_b = embeddings(other_word)
    similarity = Dot(name="Cosine-Similarity", axes=2, normalize=True)([embedded_a, embedded_b])

    # create the Keras model
    k_model = Model(inputs=[valid_word, other_word], outputs=similarity)

    def get_similarity(valid_word_idx, vocab_size):
        similarities = np.zeros((vocab_size,))
        in_arr1 = np.zeros((1,))
        in_arr2 = np.zeros((1,))
        in_arr1[0,] = valid_word_idx
        for i in range(vocab_size):
            in_arr2[0,] = i
            out = k_model.predict_on_batch([in_arr1, in_arr2])
            similarities[i] = out
        return similarities

    # now run the model and get the closest words to the valid examples
    for i in range(valid_size):
        valid_word = wv.index2word[valid_examples[i]]
        top_k = 8  # number of nearest neighbors
        similarity = get_similarity(valid_examples[i], len(wv.vocab))
        nearest = (-similarity).argsort()[1:top_k + 1]
        log_str = 'Nearest to %s:' % valid_word
        for k in range(top_k):
            close_word = wv.index2word[nearest[k]]
            log_str = '%s %s,' % (log_str, close_word)
        print(log_str)

In [4]:
""" Test Keras Model """
training = False
if training:
    model = train()
else:
    model = word2vec.Word2Vec.load("word2vec-gensim.model")

embedding_matrix = create_embedding_matrix(model)
keras_model(embedding_matrix, model.wv)

Nearest to s: sixties, his, seventies, vought, janet, stanley, christine, ted,
Nearest to people: persons, residents, citizens, individuals, jews, inhabitants, americans, albanians,
Nearest to in: throughout, during, until, since, around, within, expo, one,
Nearest to its: their, the, itself, our, it, simplicity, whose, immediate,
Nearest to state: legislature, nation, territory, judiciary, government, states, sovereign, statehood,
Nearest to be: prevail, proceed, represent, imply, satisfy, suffice, invoke, yield,
Nearest to some: many, certain, various, numerous, several, those, few, these,
Nearest to was: is, were, became, had, fell, remained, came, remains,
Nearest to six: four, eight, five, seven, three, one, nine, two,
Nearest to often: sometimes, frequently, usually, generally, typically, commonly, occasionally, normally,
Nearest to states: nations, emirates, kingdom, provinces, parcel, methodist, loyalists, wallachia,
Nearest to that: which, actually, what, therefore, whatever, 

In [5]:
""" Test Gensim """
training = False
if training:
    model = train()
else:
    model = word2vec.Word2Vec.load("word2vec-gensim.model")

# get the word vector of "the"
print(model.wv['the'])

# get the most common words
print(model.wv.index2word[0], model.wv.index2word[1], model.wv.index2word[2])

# get the least common words
vocab_size = len(model.wv.vocab)
print(model.wv.index2word[vocab_size - 1], model.wv.index2word[vocab_size - 2], model.wv.index2word[vocab_size - 3])

# find the index of the 2nd most common word ("of")
print('Index of "of" is: {}'.format(model.wv.vocab['of'].index))

# some similarity fun
print(model.wv.similarity('woman', 'man'), model.wv.similarity('man', 'elephant'))

# what doesn't fit?
print(model.wv.doesnt_match("green blue red zebra".split()))

[-0.22422697  0.14712758 -0.47601643  0.3790985   0.10261266 -0.8650235
 -0.31936124  0.5156032  -0.98320854 -0.789132    0.46489877 -0.82984257
  0.09237605 -0.08166566 -0.07527041 -0.85539114  0.8185313   0.27993223
  0.5104366   0.18904924  0.5679309   0.5547837   0.3610671   0.73984617
 -0.5218881   0.11752108 -0.6944164   0.79219466  0.5284484   0.5948236
 -0.3011668   0.14271732  0.3743451   0.455225    0.77808815 -0.7987459
 -0.7720395  -0.0787911   1.048541   -0.32106057 -0.21548207  0.20202452
 -0.44003892 -1.641467   -0.74306124  0.30159307 -0.02842805  0.64166474
 -1.4520941   0.46092674  0.32494962  0.15518251  0.9517272   0.71376425
 -0.8646511   0.1489159  -0.5240067   0.19802691  0.5452466   0.32581544
  0.2240663  -0.11205453 -0.67169976  1.0156052  -0.8727423  -0.3247499
 -0.20645174 -0.3241183   0.05699601 -0.59548783  0.5271353   0.94990444
 -0.06797419 -0.53946453  1.166609   -0.26356938  0.62137777  0.4043672
 -0.4871116  -0.05922779  0.25737008 -0.5965672  -0.2269

/Users/kadriansyah/anaconda3/envs/recommender/lib/python3.7/site-packages/gensim/models/keyedvectors.py:877: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)
